In [50]:
# import Libraries 
from scipy.sparse import csr_matrix
import pandas as pd
import plotly.express as px
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import graphviz 
from sklearn.feature_selection import SelectKBest , f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets, ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import max_error,mean_squared_error,mean_absolute_error,mean_squared_log_error,r2_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
# from sklearn.inspection import DecisionBoundaryDisplay
import numpy as np
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report,confusion_matrix
import os
import pickle
import plotly.graph_objs as go
import plotly.offline as pyo

In [53]:
def report(y_true,X_test, model_name , name, thresholds=None):
    os.makedirs(name, exist_ok=True)
    filename = os.path.join(name, name+'.sav')
    pickle.dump(model_name, open(filename, 'wb'))
    y_pred = model_name.predict(X_test)
    # Print model name
    
    print("Model: {}".format(name))
    print("========================")
    
    if hasattr(model_name, "predict_proba"):
        y_prob = model_name.predict_proba(X_test)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_true, y_prob)
        indx = np.argmax(tpr-fpr)
        threshold = thresholds[indx]
   
    else:
        threshold = [0.5]
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        y_prob = y_pred

    younY = y_prob >= threshold
    younY = younY.astype(int)

    accuracy = accuracy_score(y_true, younY)
    balancedAcc = balanced_accuracy_score(y_true, younY)
    f1metric =  f1_score(y_true, younY)
    recalll = recall_score(y_true, younY)
    precision = precision_score(y_true, younY)

    print("Threshold:", threshold)
    print("========================")
    # Calculate sensitivity and specificity
    cm = confusion_matrix(y_true, younY)
    tp = cm[0, 0]
    fn = cm[0, 1]
    fp = cm[1, 0]
    tn = cm[1, 1]

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Print the results
    auc = roc_auc_score(y_test, y_prob)
    
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("========================")
    print('precision :', precision)
    print('accuracy :', accuracy)
    print('balancedAcc :', balancedAcc)
    print('recalll :', recalll)
    print('f1metric :', f1metric)
    print('AUC:', auc)
    print("========================")

    print(classification_report(y_true, younY))
    
    
    
    result_file_name = os.path.join(name, "results.txt")
    with open(result_file_name, 'w') as f:
        f.write("Threshold: {}\n".format(threshold))
        f.write("========================\n")
        f.write("Sensitivity: {}\n".format(sensitivity))
        f.write("Specificity: {}\n".format(specificity))
        f.write("========================\n")
        f.write("Precision: {}\n".format(precision))
        f.write("Accuracy: {}\n".format(accuracy))
        f.write("Recall: {}\n".format(recalll))
        f.write("F1 Score: {}\n".format(f1metric))
        f.write("========================\n")
        f.write("AUC: {}\n".format(auc))
        f.write("========================\n")
        f.write("Classification Report:\n{}\n".format(classification_report(y_test, younY)))
    plot_file_name = os.path.join(name, "ROC_plot")
    trace1 = go.Scatter(x=fpr, y=tpr, name='ROC curve (area = %0.2f)' % auc)
    trace2 = go.Scatter(x=[0, 1], y=[0, 1], line=dict(dash='dash'), name='Random guess')
    fig = go.Figure(data=[trace1, trace2])
    fig.update_layout(
        title="{} - ROC".format(name),
        title_x=0.5,
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        legend=dict(x=0.65, y=0.09, bordercolor="Black", borderwidth=2),
        width=1200,
        height=800)
    pyo.plot(fig, filename=plot_file_name, auto_open=False)
    # Save the plot in png format
    plot_file_name = os.path.join(name, "ROC_plot.png")
    fig.write_image(plot_file_name,width=950, height=750)
    df = pd.DataFrame({
        'False Positive Rate': fpr,
        'True Positive Rate': tpr
    }, index=thresholds)
    df.index.name = "Thresholds"
    df.columns.name = "Rate"
    
    fig_thresh = px.imshow(
        df,
        x=df.columns,
        y=df.index,
        title="TPR and FPR at every threshold",
        color_continuous_scale='YlOrRd'
    )
    plot_thresh_file_name = os.path.join(name, "TPR_FPR_heatmap.html")
    fig_thresh.write_html(plot_thresh_file_name)
    plot_thresh_png_file_name = os.path.join(name, "TPR_FPR_heatmap.png")
    fig_thresh.write_image(plot_thresh_png_file_name)
    
    fig_thresh = px.line(df, title='TPR and FPR at every threshold', width=950, height=750)
    fig_thresh.update_layout(title='TPR and FPR at every threshold', title_x=0.5)
    fig_thresh.update_yaxes(scaleanchor="x", scaleratio=1)
    fig_thresh.update_xaxes(range=[0, 1], constrain='domain')
    
    
    thresh_plot_file_name = os.path.join(name, "TPR_FPR_plot.html")
    fig_thresh.write_html(thresh_plot_file_name)
    fig_thresh.write_image(os.path.join(name, "TPR_FPR_plot.png"))
    
    
    cm_file_name = os.path.join(name, "confusion_matrix.png")
    cm_fig = go.Figure(data=go.Heatmap(z=cm, x=['Predicted: 0', 'Predicted: 1'], y=['Actual: 0', 'Actual: 1'], colorscale="Blues"))
    for i in range(len(cm)):
        for j in range(len(cm)):
            cm_fig.add_annotation(x=j, y=i, text=str(cm[i][j]), showarrow=False, font=dict(color='gray', size=20))
            
    cm_fig.update_layout(
        title="Confusion Matrix for {}".format(name),
        title_x=0.5,
        xaxis_title="Predicted label",
        yaxis_title="True label",
        width=950,
        height=750
    )
    cm_fig.write_image(cm_file_name)
    cm_file_name = os.path.join(name, "confusion_matrix.html")
    
    fig2 = go.Figure(data=go.Heatmap(z=cm, x=["Predicted Negative", "Predicted Positive"], y=["Actual Negative", "Actual Positive"], colorscale="Blues"))
    fig2.update_layout(
        title="Confusion Matrix for {}".format(name),
        xaxis_title="Predicted label",
        yaxis_title="True label",
        width=1600,
        height=1200
    )
    pyo.plot(fig2, filename=cm_file_name, auto_open=False)
    print("Results and plot saved in folder:", name)

    print("Results saved in {}".format(name))

    
    
    
    #return accuracy, balancedAcc, f1metric, recalll, precision, sensitivity, specificity

In [2]:
X = pd.read_csv('/raid-18/CHS/jakeluo/share/notebook/ataleban@uwm.edu/Panc_/X_Cleared_Balance1.5.csv')  
Y = pd.read_csv('/raid-18/CHS/jakeluo/share/notebook/ataleban@uwm.edu/Panc_/Y_Cleared_Balance1.5.csv')

In [3]:
y1 = Y['0']
xe = X
xe = xe.drop(['Unnamed: 0'], axis=1)


for i in xe.columns:
    if i == 'age':
        continue
    xe[i] = xe[i].astype('category')
    
y1 = y1.astype('category')

In [24]:
X_t = SelectKBest(chi2,k=285)
X_t = X_t.fit_transform(xe,y1)
X_train, X_test, y_train, y_test = train_test_split(X_t, y1, test_size=0.2, random_state=42)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(xe, y1, test_size=0.2, random_state=42)

In [54]:
models_and_grids = {
    "AdaBoostClassifier": {
        'K' : 27,
        "model": AdaBoostClassifier(),
        "param_grid": {
            'n_estimators': [50, 100, 200],
            'learning_rate': [
                0.01, 0.1, 1]
        }
    },
    "RidgeClassifierCV": {
        'K' : 285,
        "model": RidgeClassifierCV(),
        "param_grid": {
            #'alphas': [[0.1, 1.0, 10.0]]
        }
    },
    "RandomForestClassifier": {
        'K' : 24,
        "model": RandomForestClassifier(),
        "param_grid": {
            'n_estimators': [10, 50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    },
    "GradientBoostingClassifier": {
        'K' : 23,
        "model": ensemble.GradientBoostingClassifier(),
        "param_grid": {
            'n_estimators': [100, 200, 300],
            'learning_rate': [0.01, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'subsample': [0.8, 0.9, 1.0]
        }
    },
    "KNeighborsClassifier": {
        'K' : 265,
        "model": KNeighborsClassifier(),
        "param_grid": {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan', 'minkowski']
        }
    },
    "MLPClassifier": {
        'K' : 632,
        "model": MLPClassifier(),
        "param_grid": {
            'hidden_layer_sizes': [(50,), (100,), (50, 50)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive'],
            'max_iter': [200, 300]
        }
    }
}

In [42]:
models_and_grids.items()

dict_items([('AdaBoostClassifier', {'K': 27, 'model': AdaBoostClassifier(), 'param_grid': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}}), ('RidgeClassifierCV', {'K': 285, 'model': RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ])), 'param_grid': {}}), ('RandomForestClassifier', {'K': 24, 'model': RandomForestClassifier(), 'param_grid': {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}}), ('GradientBoostingClassifier', {'K': 23, 'model': GradientBoostingClassifier(), 'param_grid': {'n_estimators': [100, 200, 300], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'subsample': [0.8, 0.9, 1.0]}}), ('KNeighborsClassifier', {'K': 265, 'model': KNeighborsClassifier(), 'param_grid': {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan', 'minkowski']}}), ('MLPClassifier'

In [56]:
# Perform GridSearchCV for each model and store the best model and parameters
best_models = {}

for model_name, model_info in models_and_grids.items():

    print(f"Performing GridSearchCV for {model_name}...")
    
    X_t = SelectKBest(chi2,k=model_info['K'])
    X_t = X_t.fit_transform(xe,y1)
    X_train, X_test, y_train, y_test = train_test_split(X_t, y1, test_size=0.2, random_state=42)

    grid_search = GridSearchCV(estimator=model_info["model"], param_grid=model_info["param_grid"], cv=5, n_jobs=-1, scoring='f1')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    
    # Train the best model for this algorithm on the entire training set
    best_estimator = grid_search.best_estimator_
    best_estimator.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = best_estimator.predict(X_test)
    
    # Calculate MSE for the best model
    accuracy = accuracy_score(y_test, y_pred)
    balancedAcc = balanced_accuracy_score(y_test, y_pred)
    f1metric =  f1_score(y_test, y_pred)
    recalll = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    
    
    #Model Reporting
    SGDclf = model_info["model"].set_params(**best_params)
    SGDclf.fit(X_train, y_train)
    y_pred = SGDclf.predict(X_test)
    report(y_test,X_test, SGDclf,model_name)

    best_models[model_name] = {
        "model": best_estimator,
        "params": best_params,
        "score": best_score,
        "f1metric": f1metric
    }
    print(f"Best parameters for {model_name}: {best_params}, f1metric score: {best_score}, f1metric: {f1metric}")


Performing GridSearchCV for AdaBoostClassifier...
Model: AdaBoostClassifier
Threshold: 0.499356589552662
Sensitivity: 0.875
Specificity: 0.808695652173913
precision : 0.8074191002367798
accuracy : 0.8489275722723034
balancedAcc : 0.8418478260869565
recalll : 0.808695652173913
f1metric : 0.8080568720379145
AUC: 0.921079221473466
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1952
           1       0.81      0.81      0.81      1265

    accuracy                           0.85      3217
   macro avg       0.84      0.84      0.84      3217
weighted avg       0.85      0.85      0.85      3217



/raid-18/CHS/jakeluo/envs/NIS2016/lib/python3.9/site-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `AdaBoostClassifier/ROC_plot` didn't end with .html. Adding .html to the end of your file.



Results and plot saved in folder: AdaBoostClassifier
Results saved in AdaBoostClassifier
Best parameters for AdaBoostClassifier: {'learning_rate': 1, 'n_estimators': 200}, f1metric score: 0.7963347571391269, f1metric: 0.7993138936535162
Performing GridSearchCV for RidgeClassifierCV...
Model: RidgeClassifierCV
Threshold: [0.5]
Sensitivity: 0.9953893442622951
Specificity: 0.7762845849802371
precision : 0.9909182643794148
accuracy : 0.9092322039166926
balancedAcc : 0.885836964621266
recalll : 0.7762845849802371
f1metric : 0.8705673758865248
AUC: 0.885836964621266
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1952
           1       0.99      0.78      0.87      1265

    accuracy                           0.91      3217
   macro avg       0.93      0.89      0.90      3217
weighted avg       0.92      0.91      0.91      3217



/raid-18/CHS/jakeluo/envs/NIS2016/lib/python3.9/site-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `RidgeClassifierCV/ROC_plot` didn't end with .html. Adding .html to the end of your file.



Results and plot saved in folder: RidgeClassifierCV
Results saved in RidgeClassifierCV
Best parameters for RidgeClassifierCV: {}, f1metric score: 0.8846101302541085, f1metric: 0.8705673758865248
Performing GridSearchCV for RandomForestClassifier...
Model: RandomForestClassifier
Threshold: 0.34454106649770777
Sensitivity: 0.8140368852459017
Specificity: 0.8561264822134388
precision : 0.7489626556016598
accuracy : 0.8305875038856078
balancedAcc : 0.8350816837296702
recalll : 0.8561264822134388
f1metric : 0.798967170785688
AUC: 0.9180018061945183
              precision    recall  f1-score   support

           0       0.90      0.81      0.85      1952
           1       0.75      0.86      0.80      1265

    accuracy                           0.83      3217
   macro avg       0.82      0.84      0.83      3217
weighted avg       0.84      0.83      0.83      3217



...

Results and plot saved in folder: GradientBoostingClassifier
Results saved in GradientBoostingClassifier
Best parameters for GradientBoostingClassifier: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200, 'subsample': 0.8}, f1metric score: 0.815948137950161, f1metric: 0.8011894647408666
Performing GridSearchCV for KNeighborsClassifier...
Model: KNeighborsClassifier
Threshold: 0.5580127007844602
Sensitivity: 0.7366803278688525
Specificity: 0.433201581027668
precision : 0.5160075329566854
accuracy : 0.6173453528131799
balancedAcc : 0.5849409544482602
recalll : 0.433201581027668
f1metric : 0.47099269445638153
AUC: 0.6161929793300072
              precision    recall  f1-score   support

           0       0.67      0.74      0.70      1952
           1       0.52      0.43      0.47      1265

    accuracy                           0.62      3217
   macro avg       0.59      0.58      0.59      3217
weighted avg       0.61      0.62 

/raid-18/CHS/jakeluo/envs/NIS2016/lib/python3.9/site-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `KNeighborsClassifier/ROC_plot` didn't end with .html. Adding .html to the end of your file.



Results and plot saved in folder: KNeighborsClassifier
Results saved in KNeighborsClassifier
Best parameters for KNeighborsClassifier: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}, f1metric score: 0.49679293313253386, f1metric: 0.4768595041322313
Performing GridSearchCV for MLPClassifier...
Model: MLPClassifier
Threshold: 0.08556102215954742
Sensitivity: 0.8632172131147541
Specificity: 0.7707509881422925
precision : 0.785024154589372
accuracy : 0.8268573204849239
balancedAcc : 0.8169841006285232
recalll : 0.7707509881422925
f1metric : 0.7778220981252493
AUC: 0.8896488045098165
              precision    recall  f1-score   support

           0       0.85      0.86      0.86      1952
           1       0.79      0.77      0.78      1265

    accuracy                           0.83      3217
   macro avg       0.82      0.82      0.82      3217
weighted avg       0.83      0.83      0.83      3217



/raid-18/CHS/jakeluo/envs/NIS2016/lib/python3.9/site-packages/plotly/offline/offline.py:562: UserWarning:

Your filename `MLPClassifier/ROC_plot` didn't end with .html. Adding .html to the end of your file.



Results and plot saved in folder: MLPClassifier
Results saved in MLPClassifier
Best parameters for MLPClassifier: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive', 'max_iter': 200, 'solver': 'adam'}, f1metric score: 0.6716341480293927, f1metric: 0.6810810810810811
